고양이 질병관련 챗봇
(챗봇,워드클라우드//이후에 다른걸 추가해보기 일단 두개부터)

챗봇

워드클라우드

In [32]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from fuzzywuzzy import process
import re

# CSV 파일 로드 (경로를 실제 파일 경로로 변경해주세요)
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# Symptoms와 Description을 합쳐서 inputs 컬럼 생성
#df['inputs'] = df['Symptoms'] + " " + df['Description']
df['inputs'] = df['Symptoms'].fillna('') + " " + df['Description'].fillna('')


# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df['inputs']))

# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

# ChatOllama 모델 초기화 (temperature 값 조정)
#llm = ChatOllama(model="gemma2", temperature=0.0)
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None


# Question Generation Prompt
question_generator_template = """
이전 대화 내역과 새로운 사용자 질문이 주어졌을 때, 검색을 위한 독립적인 질문으로 바꿔서 생성해주세요.

이전 대화 내역:
{chat_history}

새로운 사용자 질문:
{question}

독립적인 질문:
"""
QUESTION_GENERATOR_PROMPT = PromptTemplate(input_variables=["chat_history", "question"], template=question_generator_template)

# Combine Documents Prompt (한국어 지시 강화 및 형식 명확화)
combine_documents_template = """
당신은 경험 많은 고양이 수의사입니다. **모든 답변은 한국어만 사용하여, 어색함이 없도록 매우 자연스럽게 작성해야 합니다. 존댓말을 사용하여 정중하게 답변해야 합니다.** 외국어나 어색한 한국어 표현을 절대 사용하지 않도록 주의하십시오.

사용자의 질문에 대해 다음과 같은 방식으로 답변해야 합니다.

1.  질문에 대한 직접적인 답변을 **한국어로** 제공합니다.
2.  필요한 경우, 추가적인 질문을 통해 상황을 명확히 파악하려고 노력해야 합니다. 예를 들어, 증상의 기간, 심각성, 다른 동반 증상 등을 **한국어로, 부드럽게** 물어볼 수 있습니다. (예: "혹시 언제부터 그러셨나요?", "다른 불편한 점은 없으신가요?")
3.  가능한 원인 질병을 언급하고, 각 질병에 대한 간략한 설명을 **한국어로, 이해하기 쉽게** 제공합니다.
4.  집에서 할 수 있는 조치와 동물병원 방문이 필요한 경우를 명확하게 구분하여 **한국어로, 친절하게** 안내합니다.
5.  절대 진단이나 처방을 내리지 않고, 반드시 동물병원에 방문하여 정확한 진료를 받을 것을 **한국어로, 정중하게** 권장해야 합니다.

검색된 문서:
{context}

사용자 질문:
{question}

답변:
"""
COMBINE_DOCUMENTS_PROMPT = PromptTemplate(input_variables=["context", "question"], template=combine_documents_template)

# Chain 생성
question_generator = LLMChain(llm=llm, prompt=QUESTION_GENERATOR_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=COMBINE_DOCUMENTS_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    return_source_documents=True,
)

chat_history = []

# 입력한 채팅 공백이나 특수문자 예외처리
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace(" ", "")
    return text.lower()

# 질병 정보 검색 함수
def search_disease_info(message):
    best_match = process.extractOne(message, df['inputs'])
    if best_match and best_match[1] > 80:
        disease_info = df.loc[df['inputs'] == best_match[0]]
        disease_name = disease_info['Disease'].values[0]
        symptoms = disease_info['Symptoms'].values[0]
        description = disease_info['Description'].values[0]
        emergency = disease_info['Emergency'].values[0]
        return f"질병: {disease_name}\n증상: {symptoms}\n설명: {description}\n응급상황 여부: {emergency}"
    return "해당 질문에 대한 답변을 찾을 수 없습니다."

# 고양이 질병에 대한 정보 제공하는 채팅 함수
def chat(message, history):
    cleaned_message = clean_text(message)
    disease_info = search_disease_info(message)
    return disease_info

# Conversational Chat 함수 구현 (chat_history 관리)
#def conversational_chat(message, history):
#    global chat_history
#    result = qa({"question": message, "chat_history": chat_history})
#    chat_history = [(message, result["answer"])]
#    return result["answer"]

def conversational_chat(message, history=[]):
    result = qa({"question": message, "chat_history": history})
    history.append((message, result["answer"]))
    return result["answer"]


# Gradio 인터페이스 설정 (CSS 및 components 활용)
with gr.Blocks(css="""
.chat-interface-container {
    height: 650px;
    max-height: 800px;
    overflow-y: auto;
    border: 3px solid #7193BD;
    padding: 15px;
    border-radius: 15px;
}
.message {
    background-color: #F1E6BF !important;
    white-space: pre-wrap !important; /* 긴 텍스트 줄바꿈 처리 */
}
.message.user {
    background-color: #F1E6BF !important;
    padding: 10px;
    border-radius: 15px;
}
.message.ai {
    background-color: #BED8EB !important;
    padding: 10px;
    border-radius: 15px;
}
.chat-title {
    font-size: 35px !important; /* 글씨 크기 조정 */
    color: #93B9D6 !important; /* 글씨 색상 조정 - 여기서 세미콜론 앞 공백 제거 */
    text-align: center;
    margin-bottom: 20px;
    font-weight: bold !important; /* 글자 두께 조정 */
}
.description { 
    text-align: right !important; /* 텍스트 오른쪽 정렬 */ 
    color: #6D6875 !important;
    font-weight: bold !important;
    margin-bottom: 20px; /* 추가로 간격 조정 */
}
""") as demo:
    with gr.Tabs():
        with gr.TabItem(" 고양이 질병 AI 챗봇 "):
            chat_demo = gr.ChatInterface(fn=chat, examples=["침 흘림, 식사 거부, 입 냄새", "잦은 배변, 물 같은 변, 탈수", "구토, 설사, 체중 감소."], title="고양이 질병 AI 챗봇", description="고양이의 질병과 증상에 대해 질문하면, 질병 정보와 응급상황 여부를 AI가 알려줍니다.")
            chat_demo

        with gr.TabItem(" 고양이 질병 자유형 AI 챗봇 "):
            with gr.Column(elem_classes=["chat-interface-container"]):
                # Div 추가하여 스타일 적용
                gr.HTML("""
                    <div class="chat-title">고양이 질병 자가진단 챗봇</div>
                    <div class="description">고양이 질병에 대해 자유롭게 질문하고 답변을 받아보세요.</div>
                """)
                chatbot = gr.ChatInterface(
                    fn=conversational_chat,
                    examples=[
                        "고양이가 식사 거부하고 침을 많이 흘려요.",
                        "고양이가 복부에 이상이 있는 것 같아요.",
                        "고양이의 증상에 대해 설명해주세요.",
                        "고양이 범백혈구 감소증의 원인은 무엇인가요?",
                        "고양이에게 흔한 질병은 무엇인가요?"
                    ]
                ) # components 인자 제거

demo.launch(server_port=7861, server_name="0.0.0.0")


c:\HJY310\.venv\lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Tap탭

In [82]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from fuzzywuzzy import process
import re

# CSV 파일 로드 (경로를 실제 파일 경로로 변경해주세요)
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# Symptoms와 Description을 합쳐서 inputs 컬럼 생성
df['inputs'] = df['Symptoms'].fillna('') + " " + df['Description'].fillna('')

# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df['inputs']))

# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

# ChatOllama 모델 초기화 (temperature 값 조정)
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None

# Question Generation Prompt
question_generator_template = """
이전 대화 내역과 새로운 사용자 질문이 주어졌을 때, 검색을 위한 독립적인 질문으로 바꿔서 생성해주세요.

이전 대화 내역:
{chat_history}

새로운 사용자 질문:
{question}

독립적인 질문:
"""
QUESTION_GENERATOR_PROMPT = PromptTemplate(input_variables=["chat_history", "question"], template=question_generator_template)

# Combine Documents Prompt (한국어 지시 강화 및 형식 명확화)
combine_documents_template = """
당신은 경험 많은 고양이 수의사입니다. **모든 답변은 한국어만 사용하여, 어색함이 없도록 매우 자연스럽게 작성해야 합니다. 존댓말을 사용하여 정중하게 답변해야 합니다.** 외국어나 어색한 한국어 표현을 절대 사용하지 않도록 주의하십시오.

사용자의 질문에 대해 다음과 같은 방식으로 답변해야 합니다.

1.  질문에 대한 직접적인 답변을 **한국어로** 제공합니다.
2.  필요한 경우, 추가적인 질문을 통해 상황을 명확히 파악하려고 노력해야 합니다. 예를 들어, 증상의 기간, 심각성, 다른 동반 증상 등을 **한국어로, 부드럽게** 물어볼 수 있습니다. (예: "혹시 언제부터 그러셨나요?", "다른 불편한 점은 없으신가요?")
3.  가능한 원인 질병을 언급하고, 각 질병에 대한 간략한 설명을 **한국어로, 이해하기 쉽게** 제공합니다.
4.  집에서 할 수 있는 조치와 동물병원 방문이 필요한 경우를 명확하게 구분하여 **한국어로, 친절하게** 안내합니다.
5.  절대 진단이나 처방을 내리지 않고, 반드시 동물병원에 방문하여 정확한 진료를 받을 것을 **한국어로, 정중하게** 권장해야 합니다.
6.  사용자가 걱정하거나 불안한 감정을 느낄 수 있기 때문에, **위로**와 **안심**을 주는 말을 포함하여 사용자가 더 편안하게 느끼도록 합니다.

검색된 문서:
{context}

사용자 질문:
{question}

답변:
"""
COMBINE_DOCUMENTS_PROMPT = PromptTemplate(input_variables=["context", "question"], template=combine_documents_template)

# Chain 생성
question_generator = LLMChain(llm=llm, prompt=QUESTION_GENERATOR_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=COMBINE_DOCUMENTS_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    return_source_documents=True,
)

chat_history = []

# 입력한 채팅 공백이나 특수문자 예외처리
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace(" ", "")
    return text.lower()

# 질병 정보 검색 함수
def search_disease_info(message):
    best_match = process.extractOne(message, df['inputs'])
    if best_match and best_match[1] > 80:
        disease_info = df.loc[df['inputs'] == best_match[0]]
        disease_name = disease_info['Disease'].values[0]
        symptoms = disease_info['Symptoms'].values[0]
        description = disease_info['Description'].values[0]
        emergency = disease_info['Emergency'].values[0]
        return f"질병: {disease_name}\n증상: {symptoms}\n설명: {description}\n응급상황 여부: {emergency}"
    return "해당 질문에 대한 답변을 찾을 수 없습니다."

# 고양이 질병에 대한 정보 제공하는 채팅 함수
def chat(message, history):
    cleaned_message = clean_text(message)
    disease_info = search_disease_info(message)
    return disease_info

# Conversational Chat 함수 구현 (chat_history 관리)
def conversational_chat(message, history=[]):
#def conversational_chat(message, history):
    result = qa({"question": message, "chat_history": history})
    history.append((message, result["answer"]))
    return result["answer"]

#def conversational_chat(message, history):
    # 새로운 질문이 추가되기 전에 기존 history에서 동일한 질문이 있는지 체크
 #   if message not in [m for m, _ in history]:
 #       history.append((message, ""))
    
    # QA 수행
  #  result = qa({"question": message, "chat_history": history})
    
    # 새로운 응답을 history에 추가
  #  history[-1] = (message, result['answer'])
    
  #  return result['answer']

# 세 번째 탭용 함수
custom_prompt = """
당신은 고양이 질병에 대해 전문적인 수의사 역할을 합니다.
아래 증상에 기반하여 가능한 질병과 관련 정보를 작성하세요:

증상: {symptom}

답변은 반드시 한국어로 작성하세요.
"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["symptom"], template=custom_prompt)
chain = LLMChain(llm=llm, prompt=PROMPT_TEMPLATE)

def predict_from_form(symptom, age, duration):
    prompt = f"""
    고양이 나이: {age}
    증상 발생 기간: {duration}일
    증상: {symptom}
    
    위 정보를 기반으로 가능한 질병과 대처 방법을 작성하세요.
    """
    response = chain.run(symptom=prompt)
    return response

# 드롭다운 기반 예측 함수
def predict_disease_from_card(symptom):
    best_match = process.extractOne(symptom, df['Symptoms'].dropna())
    if best_match and best_match[1] > 80:
        disease_info = df.loc[df['Symptoms'] == best_match[0]]
        disease_name = disease_info['Disease'].values[0]
        symptoms = disease_info['Symptoms'].values[0]
        description = disease_info['Description'].values[0]
        return f"질병: {disease_name}\n증상: {symptoms}\n설명: {description}"
    return "해당 증상에 대한 질병 정보를 찾을 수 없습니다."

# Gradio 인터페이스 설정
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs():
        with gr.TabItem(" 고양이 질병 AI 챗봇 "):
            chat_demo = gr.ChatInterface(
                fn=chat, 
                examples=[
                    "침 흘림, 식사 거부, 입 냄새", 
                    "잦은 배변, 물 같은 변, 탈수", 
                    "구토, 설사, 체중 감소."
                ],
                title="고양이 질병 AI 챗봇",
                description="고양이의 질병과 증상에 대해 질문하면, 질병 정보와 응급상황 여부를 AI가 알려줍니다."
            )
            chat_demo

        with gr.TabItem(" 고양이 질병 자유형 AI 챗봇 "):
            with gr.Column():
                gr.HTML("""
                    <div style="text-align: center; font-size: 24px; color: #7193BD; font-weight: bold; margin-bottom: 15px;">고양이 질병 자가진단 챗봇</div>
                """)
                chatbot = gr.ChatInterface(
                    fn=conversational_chat,
                    examples=[
                        "고양이가 식사 거부하고 침을 많이 흘려요.",
                        "고양이가 복부에 이상이 있는 것 같아요.",
                        "고양이의 증상에 대해 설명해주세요.",
                        "고양이 범백혈구 감소증의 원인은 무엇인가요?",
                        "고양이에게 흔한 질병은 무엇인가요?"
                    ]
                )

        with gr.TabItem(" 고양이 질병 예측 - 양식 기반 "):
            gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 양식 기반")
            gr.Markdown("### 고양이의 상태를 자세히 입력하세요.")
            symptom_input = gr.Textbox(label="증상 입력", placeholder="예: 고양이가 밥을 안 먹고 구토를 해요.")
            age_input = gr.Number(label="고양이 나이 (년)", value=3)
            duration_input = gr.Number(label="증상 지속 기간 (일)", value=1)

            output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)
            submit_button = gr.Button("진단 요청")
            submit_button.click(fn=predict_from_form, inputs=[symptom_input, age_input, duration_input], outputs=output_box)

        with gr.TabItem(" 고양이 질병 예측 - 드롭다운 기반 "):
            gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 드롭다운 기반")
            gr.Markdown("### 고양이의 증상 중 하나를 드롭다운에서 선택하세요.")
            
            # 증상 목록을 드롭다운에 추가
            symptoms = df['Symptoms'].dropna().unique().tolist()
            
            symptom_dropdown = gr.Dropdown(choices=symptoms, label="증상 선택", interactive=True)
            output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)

            submit_button = gr.Button("진단 요청")
            submit_button.click(fn=predict_disease_from_card, inputs=symptom_dropdown, outputs=output_box)

demo.launch(server_port=7861, server_name="0.0.0.0")


c:\HJY310\.venv\lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [68]:
demo.close()

Closing server running on port: 7861


In [49]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# CSV 파일 로드 (경로를 실제 파일 경로로 변경)
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# Symptoms와 Description 합치기
df['inputs'] = df['Symptoms'].fillna('') + " " + df['Description'].fillna('')

# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df['inputs']))

# 임베딩 및 벡터 데이터베이스 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(texts, embeddings)

# ChatOllama 모델 초기화
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None

# 사용자 질문과 질병 정보를 바탕으로 답변 생성
custom_prompt = """
당신은 고양이 질병에 대해 전문적인 수의사 역할을 합니다. 
사용자가 제공한 증상을 바탕으로 다음 정보를 작성하세요:

1. 해당 증상에 대해 가능한 질병 목록을 생성하세요.
2. 각 질병의 증상과 간략한 설명을 추가하세요.
3. 응급상황인지 여부를 사용자에게 설명하고 필요한 조치를 안내하세요.
4. 반드시 한국어로 작성하세요.

사용자 입력: {input}
"""
PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["input"], 
    template=custom_prompt
)
chain = LLMChain(llm=llm, prompt=PROMPT_TEMPLATE)

# Gradio 인터페이스 함수
def predict_disease_info(user_input):
    """
    사용자가 입력한 고양이 증상에 대해 LLM을 이용해 답변 생성.
    """
    # LLM을 사용해 답변 생성
    response = chain.run(input=user_input)
    return response

# Gradio 인터페이스 설정
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🐾 고양이 질병 진단 서비스")
    gr.Markdown("### 고양이의 증상에 대해 입력하면, 가능한 질병 정보와 조언을 제공합니다.")
    
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="고양이 증상 입력", placeholder="예: 고양이가 밥을 안 먹고 구토를 해요.")
        with gr.Column():
            output_text = gr.Textbox(label="진단 결과", interactive=False)
    
    # 버튼 생성
    submit_button = gr.Button("질병 예측하기")
    submit_button.click(fn=predict_disease_info, inputs=user_input, outputs=output_text)

# 앱 실행
demo.launch(server_port=7861, server_name="0.0.0.0")


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\human-28\AppData\Local\Temp\ipykernel_12896\1010872933.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input=user_input)


In [51]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 데이터 로드
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# ChatOllama 초기화
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None

# Prompt 설정
custom_prompt = """
당신은 고양이 질병에 대해 전문적인 수의사 역할을 합니다.
아래 증상에 기반하여 가능한 질병과 관련 정보를 작성하세요:

증상: {symptom}

답변은 반드시 한국어로 작성하세요.
"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["symptom"], template=custom_prompt)
chain = LLMChain(llm=llm, prompt=PROMPT_TEMPLATE)

# Gradio 함수
def predict_disease_from_card(selected_symptom):
    response = chain.run(symptom=selected_symptom)
    return response

# Gradio UI 구성
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 카드 기반")
    gr.Markdown("### 고양이의 증상 중 하나를 선택하세요.")
    
    # 증상 선택 카드
    symptoms = df['Symptoms'].dropna().unique().tolist()
    symptom_cards = gr.Radio(symptoms, label="증상 선택", interactive=True)

    # 출력
    output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)
    submit_button = gr.Button("진단 요청")

    submit_button.click(fn=predict_disease_from_card, inputs=symptom_cards, outputs=output_box)

demo.launch(server_port=7861, server_name="0.0.0.0")


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [53]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 데이터 로드
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# ChatOllama 초기화
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None

# Prompt 설정
custom_prompt = """
당신은 고양이 질병에 대해 전문적인 수의사 역할을 합니다.
아래 증상에 기반하여 가능한 질병과 관련 정보를 작성하세요:

증상: {symptom}

답변은 반드시 한국어로 작성하세요.
"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["symptom"], template=custom_prompt)
chain = LLMChain(llm=llm, prompt=PROMPT_TEMPLATE)

def predict_from_form(symptom, age, duration):
    prompt = f"""
    고양이 나이: {age}
    증상 발생 기간: {duration}일
    증상: {symptom}
    
    위 정보를 기반으로 가능한 질병과 대처 방법을 작성하세요.
    """
    response = chain.run(symptom=prompt)
    return response

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 양식 기반")
    gr.Markdown("### 고양이의 상태를 자세히 입력하세요.")
    
    # 양식 입력
    symptom_input = gr.Textbox(label="증상 입력", placeholder="예: 고양이가 밥을 안 먹고 구토를 해요.")
    age_input = gr.Number(label="고양이 나이 (년)", value=3)
    duration_input = gr.Number(label="증상 지속 기간 (일)", value=1)

    output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)
    submit_button = gr.Button("진단 요청")

    submit_button.click(fn=predict_from_form, inputs=[symptom_input, age_input, duration_input], outputs=output_box)


demo.launch(server_port=7861, server_name="0.0.0.0")


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [54]:
demo.close()

Closing server running on port: 7861


In [84]:
import gradio as gr
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from fuzzywuzzy import process
import re

# CSV 파일 로드 (경로를 실제 파일 경로로 변경해주세요)
df = pd.read_csv("./data/cat_diseases.csv", encoding='CP949')

# Symptoms와 Description을 합쳐서 inputs 컬럼 생성
df['inputs'] = df['Symptoms'].fillna('') + " " + df['Description'].fillna('')

# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df['inputs']))

# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

# ChatOllama 모델 초기화 (temperature 값 조정)
try:
    llm = ChatOllama(model="gemma2", temperature=0.0)
except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    llm = None

# Question Generation Prompt
question_generator_template = """
이전 대화 내역과 새로운 사용자 질문이 주어졌을 때, 검색을 위한 독립적인 질문으로 바꿔서 생성해주세요.

이전 대화 내역:
{chat_history}

새로운 사용자 질문:
{question}

독립적인 질문:
"""
QUESTION_GENERATOR_PROMPT = PromptTemplate(input_variables=["chat_history", "question"], template=question_generator_template)

# Combine Documents Prompt (한국어 지시 강화 및 형식 명확화)
combine_documents_template = """
당신은 경험 많은 고양이 수의사입니다. **모든 답변은 한국어만 사용하여, 어색함이 없도록 매우 자연스럽게 작성해야 합니다. 존댓말을 사용하여 정중하게 답변해야 합니다.** 외국어나 어색한 한국어 표현을 절대 사용하지 않도록 주의하십시오.

사용자의 질문에 대해 다음과 같은 방식으로 답변해야 합니다.

1.  질문에 대한 직접적인 답변을 **한국어로** 제공합니다.
2.  필요한 경우, 추가적인 질문을 통해 상황을 명확히 파악하려고 노력해야 합니다. 예를 들어, 증상의 기간, 심각성, 다른 동반 증상 등을 **한국어로, 부드럽게** 물어볼 수 있습니다. (예: "혹시 언제부터 그러셨나요?", "다른 불편한 점은 없으신가요?")
3.  가능한 원인 질병을 언급하고, 각 질병에 대한 간략한 설명을 **한국어로, 이해하기 쉽게** 제공합니다.
4.  집에서 할 수 있는 조치와 동물병원 방문이 필요한 경우를 명확하게 구분하여 **한국어로, 친절하게** 안내합니다.
5.  절대 진단이나 처방을 내리지 않고, 반드시 동물병원에 방문하여 정확한 진료를 받을 것을 **한국어로, 정중하게** 권장해야 합니다.
6.  사용자가 걱정하거나 불안한 감정을 느낄 수 있기 때문에, **위로**와 **안심**을 주는 말을 포함하여 사용자가 더 편안하게 느끼도록 합니다.

검색된 문서:
{context}

사용자 질문:
{question}

답변:
"""
COMBINE_DOCUMENTS_PROMPT = PromptTemplate(input_variables=["context", "question"], template=combine_documents_template)

# Chain 생성
question_generator = LLMChain(llm=llm, prompt=QUESTION_GENERATOR_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=COMBINE_DOCUMENTS_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    return_source_documents=True,
)

chat_history = []

# 입력한 채팅 공백이나 특수문자 예외처리
def clean_text(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace(" ", "")
    return text.lower()

# 고양이 질병에 대한 정보 제공하는 채팅 함수
def chat(message, history):
    cleaned_message = clean_text(message)
    disease_info = search_disease_info(message)
    return disease_info

# Conversational Chat 함수 구현 (chat_history 관리)
def conversational_chat(message, history):
    # QA 수행
    result = qa({"question": message, "chat_history": history})
    
    # Gradio가 history를 관리하므로 history를 직접 업데이트하지 않음
    response = result["answer"]
    return response  # 응답만 반환

# 세 번째 탭용 함수
custom_prompt = """
당신은 고양이 질병에 대해 전문적인 수의사 역할을 합니다.
아래 증상에 기반하여 가능한 질병과 관련 정보를 작성하세요:

증상: {symptom}

답변은 반드시 한국어로 작성하세요.
"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["symptom"], template=custom_prompt)
chain = LLMChain(llm=llm, prompt=PROMPT_TEMPLATE)

def predict_from_form(symptom, age, duration):
    prompt = f"""
    고양이 나이: {age}
    증상 발생 기간: {duration}일
    증상: {symptom}
    
    위 정보를 기반으로 가능한 질병과 대처 방법을 작성하세요.
    """
    response = chain.run(symptom=prompt)
    return response

# 드롭다운 기반 예측 함수
def predict_disease_from_card(symptom):
    best_match = process.extractOne(symptom, df['Symptoms'].dropna())
    if best_match and best_match[1] > 80:
        disease_info = df.loc[df['Symptoms'] == best_match[0]]
        disease_name = disease_info['Disease'].values[0]
        symptoms = disease_info['Symptoms'].values[0]
        description = disease_info['Description'].values[0]
        return f"질병: {disease_name}\n증상: {symptoms}\n설명: {description}"
    return "해당 증상에 대한 질병 정보를 찾을 수 없습니다."

# Gradio 인터페이스 설정
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs():
        with gr.TabItem(" 고양이 질병 자유형 AI 챗봇 "):
            with gr.Column():
                gr.HTML("""
                    <div style="text-align: center; font-size: 24px; color: #7193BD; font-weight: bold; margin-bottom: 15px;">고양이 질병 자가진단 챗봇</div>
                """)
                chatbot = gr.ChatInterface(
                    fn=conversational_chat,
                    examples=[
                        "고양이가 식사 거부하고 침을 많이 흘려요.",
                        "고양이가 복부에 이상이 있는 것 같아요.",
                        "고양이의 증상에 대해 설명해주세요.",
                        "고양이 범백혈구 감소증의 원인은 무엇인가요?",
                        "고양이에게 흔한 질병은 무엇인가요?"
                    ]
                )

        with gr.TabItem(" 고양이 질병 예측 - 양식 기반 "):
            gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 양식 기반")
            gr.Markdown("### 고양이의 상태를 자세히 입력하세요.")
            symptom_input = gr.Textbox(label="증상 입력", placeholder="예: 고양이가 밥을 안 먹고 구토를 해요.")
            age_input = gr.Number(label="고양이 나이 (년)", value=3)
            duration_input = gr.Number(label="증상 지속 기간 (일)", value=1)

            output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)
            submit_button = gr.Button("진단 요청")
            submit_button.click(fn=predict_from_form, inputs=[symptom_input, age_input, duration_input], outputs=output_box)

        with gr.TabItem(" 고양이 질병 예측 - 드롭다운 기반 "):
            gr.Markdown("## 🐾 고양이 질병 예측 서비스 - 드롭다운 기반")
            gr.Markdown("### 고양이의 증상 중 하나를 드롭다운에서 선택하세요.")
            
            # 증상 목록을 드롭다운에 추가
            symptoms = df['Symptoms'].dropna().unique().tolist()
            
            symptom_dropdown = gr.Dropdown(choices=symptoms, label="증상 선택", interactive=True)
            output_box = gr.Textbox(label="질병 진단 결과", lines=10, interactive=False)

            submit_button = gr.Button("진단 요청")
            submit_button.click(fn=predict_disease_from_card, inputs=symptom_dropdown, outputs=output_box)

demo.launch(server_port=7861, server_name="0.0.0.0", share=True)


c:\HJY310\.venv\lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://7e63215a51307fd172.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [85]:
demo.close()

Closing server running on port: 7861
